In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install skorch

## Import Libraries

In [ ]:
# Importing necessary libraries
import warnings
warnings.filterwarnings("ignore")

import random
import time
import string
from datetime import datetime
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import hstack, csr_matrix
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import (ElasticNet, Lasso, BayesianRidge, LassoLarsIC, Ridge,
                                  LogisticRegression)
import sklearn.metrics
from sklearn.metrics import (confusion_matrix, roc_curve, auc, accuracy_score, 
                             f1_score, precision_score, recall_score,classification_report)
from sklearn.model_selection import (KFold, cross_val_score, train_test_split, 
                                     GridSearchCV)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (LabelEncoder, OneHotEncoder, RobustScaler, 
                                   StandardScaler, Normalizer)
from sklearn.svm import LinearSVC

import xgboost as xgb
import lightgbm as lgb
from mlxtend.regressor import StackingRegressor, StackingCVRegressor

from sklearn.impute import SimpleImputer

from imblearn.pipeline import Pipeline as imblearnPipeline


from imblearn.over_sampling import SMOTE

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from skorch import NeuralNetClassifier

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error




import joblib
import pickle
import nltk

%matplotlib inline


In [ ]:
train_values = pd.read_csv('/kaggle/input/clean-data/Clean Data/train_values.csv')
train_labels = pd.read_csv('/kaggle/input/clean-data/Clean Data/train_labels.csv')
test_values = pd.read_csv('/kaggle/input/clean-data/Clean Data/test_values.csv')
test_labels = pd.read_csv('/kaggle/input/clean-data/Clean Data/test_labels.csv')

train_values['Labels'] = train_labels['Labels']
test_values['Labels'] = test_labels['Labels']

# Combine train_values and test_values into one DataFrame
df = pd.concat([train_values, test_values], ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# Plotting the distribution of height for pre and post earthquake
plt.figure(figsize=(14, 6))

# Pre-earthquake height distribution
plt.subplot(1, 2, 1)
plt.hist(df['height_ft_pre_eq'], bins=30, color='blue', alpha=0.7)
plt.title('Height Distribution Pre-Earthquake')
plt.xlabel('Height (ft)')
plt.ylabel('Frequency')

# Post-earthquake height distribution
plt.subplot(1, 2, 2)
plt.hist(df['height_ft_post_eq'], bins=30, color='red', alpha=0.7)
plt.title('Height Distribution Post-Earthquake')
plt.xlabel('Height (ft)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df.set_index('building_id', inplace=True)

df=df.drop(['Labels','geo1','geo2','geo3'], axis=1)

In [ ]:
# Separate features and target variable
X = df.drop(['height_ft_post_eq','count_floors_post_eq'], axis=1)
y = df['height_ft_post_eq']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Data in Train dataset:', len(X_train))
print('Data in Test dataset:', len(X_test))

## Data Preprocessing

In [ ]:
# Define numeric and categorical features
num_feat = X.select_dtypes(include=['int64', 'float64']).columns
cat_feat = X.select_dtypes(include=['object']).columns

# Create the preprocessing steps for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
    ('scaler', StandardScaler())])

# Create the preprocessing steps for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_feat),
        ('cat', categorical_transformer, cat_feat)])

## XGBoost Regressor

In [ ]:
# Create the pipeline with preprocessing and XGBoost regressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42))
])

# Hyperparameter tuning with Grid Search
params = {
    'regressor__max_depth': [3, 5, 7],
    'regressor__n_estimators': [50, 100, 200]
}

grid = GridSearchCV(pipeline, param_grid=params, cv=5, n_jobs=-1, verbose=1)


start_time = time.time()


# Train the model
grid.fit(X_train, y_train)


end_time = time.time()
training_time = end_time - start_time

# Get the best model
best_xgb_model = grid.best_estimator_

In [ ]:
# Predict on the test set
y_pred = pipeline.predict(X_test)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

In [ ]:
def regression_evaluation_report(y_true, y_pred):
    """
    Prints an evaluation report for regression models.
    
    Parameters:
        y_true (array-like): True target values.
        y_pred (array-like): Predicted target values.
    """
    # Calculate metrics
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)
    p = len(y_pred[0]) if len(y_pred.shape) > 1 else 1
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))
    
    # Print metrics
    print("Regression Evaluation Report")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"R-squared (R²): {r2:.4f}")
    print(f"Adjusted R-squared: {adjusted_r2:.4f}")
    
    # Plot True vs Predicted values
    plt.figure(figsize=(8, 6))
    plt.scatter(y_true, y_pred, alpha=0.3)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], color='red', linestyle='--', linewidth=2)
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title('True vs Predicted Values')
    plt.show()
    
    # Plot Residuals
    residuals = y_true - y_pred
    plt.figure(figsize=(8, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.title('Residuals Distribution')
    plt.show()

## Evaluation

In [ ]:
regression_evaluation_report(y_test, y_pred)

In [ ]:
# Save the model
xgb_model_filename = 'height_prediction_xgboost_regressor_55_model.joblib'
joblib.dump(best_xgb_model, xgb_model_filename)

print(f'XGBoost Model saved as {xgb_model_filename}')

## Hyperparameter Tuning

In [ ]:
# Create the pipeline with preprocessing and XGBoost regressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42))
])

# Hyperparameter tuning with Grid Search
params = {
    'regressor__max_depth': [ 7, 9, 11],
    'regressor__n_estimators': [200, 300, 500],
    'regressor__learning_rate': [0.05, 0.1],
    'regressor__subsample': [ 0.8, 1.0],

}


grid = GridSearchCV(pipeline, param_grid=params, cv=5, n_jobs=-1, verbose=1)


start_time = time.time()


# Train the model
grid.fit(X_train, y_train)


end_time = time.time()
training_time = end_time - start_time

# Get the best model
best_xgb_model = grid.best_estimator_
best_params = grid.best_params_

# Print the best hyperparameters
print("Best Parameters:", best_params)

In [ ]:
# Predict on the test set
y_pred = grid.predict(X_test)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')
print('time:',training_time/60)

In [ ]:
regression_evaluation_report(y_test, y_pred)

## Random Forest Regressor

In [ ]:
# Create the pipeline with preprocessing and RandomForestRegressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Train the model
pipeline.fit(X_train, y_train)


In [ ]:
# Predict on the test set
y_pred = pipeline.predict(X_test)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

## Evaluation

In [ ]:
regression_evaluation_report(y_test, y_pred)